In [5]:
!pip install pyathena


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [6]:
import pandas as pd
from pyathena import connect
import matplotlib.pyplot as plt
import boto3
# Establish connection to Athena
conn = connect(s3_staging_dir='s3://step3.3/', region_name='us-east-2')

In [7]:
# Query veh0
query = '''
SELECT timestep_time, vehicle_co2
FROM "cs437-lab4"."cs437iot_lab4bucket"
WHERE vehicle_id = 'veh0'
ORDER BY timestep_time
LIMIT 100
'''
df = pd.read_sql(query, conn)

/var/folders/jm/rgjqkd2s5j5d309wnmpsvddr0000gn/T/ipykernel_75159/3995870251.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
Failed to execute query.
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/pyathena/common.py", line 586, in _execute
    query_id = retry_api_call(
               ^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/tenacity/__init__.py", line 378, in iter
    result = act

DatabaseError: Execution failed on sql: SELECT device_id, data.vehicle_CO2  FROM cs437iot_lab4bucket
An error occurred (AccessDeniedException) when calling the StartQueryExecution operation: You are not authorized to perform: athena:StartQueryExecution on the resource. After your AWS administrator or you have updated your permissions, please try again.
unable to rollback

In [ ]:
# CO2 emissions over time
plt.figure(figsize=(10, 5))
plt.plot(df['timestep_time'], df['vehicle_co2'], marker='o')
plt.title("Vehicle CO2 Emissions Over Time for veh0")
plt.xlabel("Time")
plt.ylabel("CO2 Emissions")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# avg CO2 and speed per vehicle over time
query = '''
SELECT
  vehicle_id,
  CAST(timestep_time AS int) AS timestep,
  AVG(vehicle_co2) AS avg_co2,
  AVG(vehicle_speed) AS avg_speed
FROM "cs437-lab4"."cs437iot_lab4bucket"
WHERE vehicle_id IS NOT NULL
GROUP BY vehicle_id, CAST(timestep_time AS int)
ORDER BY vehicle_id, timestep
LIMIT 500
'''

df = pd.read_sql(query, conn)
df.columns = df.columns.str.strip()

print(df.head())

In [ ]:
plt.figure(figsize=(12, 2))

for vehicle in df['vehicle_id'].unique():
    v_df = df[df['vehicle_id'] == vehicle]
    plt.plot(v_df['avg_speed'], v_df['avg_co2'], marker='o', label=vehicle)

plt.title("CO₂ Emissions vs Speed per Vehicle")
plt.xlabel("Average Speed")
plt.ylabel("Average CO₂ Emission")
plt.legend(title="Vehicle ID")
plt.grid(True)
plt.tight_layout()
plt.show()
